In [1]:
import os
import datetime
# import numpy as np
# import pandas as pd
import polars as pl

In [2]:
now = datetime.datetime.now()
print(now)

2025-05-24 00:47:48.758950


In [3]:
HNratio = pl.read_csv(
    "../Data/05_HNratio/HNratio_rice_250524_all.csv", 
    separator=","
)
display(HNratio)

GENEID,SRR22741460,SRR22741461,SRR22741462,SRR22741463,SRR22741468,SRR22741469,SRR22741470,SRR22741471,SRR10991576,SRR10991577,SRR10991578,SRR10991582,SRR10991583,SRR10991584,SRR8140273,SRR8140274,SRR8140275,SRR8140276,SRR8140277,SRR8140278,SRR8140285,SRR8140286,SRR8140287,SRR8140288,SRR8140289,SRR8140290,SRR10423442,SRR10423443,SRR10423444,SRR10423430,SRR10423431,SRR10423432,SRR10423433,SRR10423434,SRR10423435,SRR10423418,…,SRR15060432,SRR15060433,SRR15060434,SRR15060435,SRR15060436,SRR15060437,SRR15060438,SRR15060439,SRR15060440,SRR15060441,SRR15060442,SRR15060443,SRR15060444,SRR15060445,SRR15060446,SRR15060447,SRR15060448,SRR15060449,SRR15060450,SRR15060451,SRR15060452,SRR15060453,SRR15060454,SRR15060455,SRR15060456,SRR22854097,SRR22854098,SRR22854099,SRR22854094,SRR22854095,SRR22854096,SRR23051879,SRR23051878,SRR23051877,SRR23051876,SRR23051875,SRR23051874
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""EPlORYSAT000373610""",1.0,1.0,64.003051,64.326859,0.015755,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,188.995833,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,200.462188,1.0,1.0,1.0,0.017064,1.0,70.185139,1.284229,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373621""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373643""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373795""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""EPlORYSAT000373851""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,…,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Os12g0641100""",0.330482,0.881209,0.498074,0.510708,0.808446,0.986463,1.23701,0.690745,1.967604,2.090501,1.667619,0.663144,0.915042,1.175697,0.290975,0.287567,0.243554,0.240388,0.361402,0.367039,0.361091,0.361422,0.356863,0.362518,0.45673,0.463282,5.218504,3.569581,3.710993,2.168091,1.36272,1.019475,2.594985,1.684937,1.135803,10.381417,…,1.254025,0.520597,1.459464,0.983362,0.992884,1.093461,1.780748,1.34706,1.17272,2.118824,1.495923,1.029656,0.80882,1.615215,1.139736,0.948756,0.962316,0.721113,1.061525,1.037891,1.287411,2.775175,0.525017,2.270865,0.577094,0.639495,0.560287,0.520431,0.792163,0.735383,0.738452,1.377318,0.974089,0.60175,0.558296,0.972543,1.624633
"""Os12g0641200""",1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.34581,1.337851,0.722855,0.202596,0.310684,0.482447,2.147339,2.66812,2.470606,2.420584,1.210811,1.68822,1.782365,2.02234,0.864469,0.57654,2.820844,3.900592,1.0,1.0,1.363146,0.224637,2.561818,5.592903,0.891283,1.0,2.886571,0.399646,…,1.626003,0.554418,2.327543,0.708659,1.253904,0.747667,1.21419,1.394378,0.3072,1.

In [4]:
# set the threshold
threshold = 5

# define a function to categorize the HNratio
def categorize(x):
    if x >= threshold:
        return 'UP'
    elif x <= 1/threshold:
        return 'DOWN'
    else:
        return 'UNCHANGED'

# apply the categorize function to all columns except GENEID
result = HNratio.with_columns([
    pl.col(col).map_elements(categorize, return_dtype=pl.String).alias(f"{col}_category")
    for col in HNratio.columns if col != 'GENEID'
])

result = result.drop(
    [col for col in result.columns if not col.endswith('_category') and col != 'GENEID']
)

# display the result
display(result)

GENEID,SRR22741460_category,SRR22741461_category,SRR22741462_category,SRR22741463_category,SRR22741468_category,SRR22741469_category,SRR22741470_category,SRR22741471_category,SRR10991576_category,SRR10991577_category,SRR10991578_category,SRR10991582_category,SRR10991583_category,SRR10991584_category,SRR8140273_category,SRR8140274_category,SRR8140275_category,SRR8140276_category,SRR8140277_category,SRR8140278_category,SRR8140285_category,SRR8140286_category,SRR8140287_category,SRR8140288_category,SRR8140289_category,SRR8140290_category,SRR10423442_category,SRR10423443_category,SRR10423444_category,SRR10423430_category,SRR10423431_category,SRR10423432_category,SRR10423433_category,SRR10423434_category,SRR10423435_category,SRR10423418_category,…,SRR15060432_category,SRR15060433_category,SRR15060434_category,SRR15060435_category,SRR15060436_category,SRR15060437_category,SRR15060438_category,SRR15060439_category,SRR15060440_category,SRR15060441_category,SRR15060442_category,SRR15060443_category,SRR15060444_category,SRR15060445_category,SRR15060446_category,SRR15060447_category,SRR15060448_category,SRR15060449_category,SRR15060450_category,SRR15060451_category,SRR15060452_category,SRR15060453_category,SRR15060454_category,SRR15060455_category,SRR15060456_category,SRR22854097_category,SRR22854098_category,SRR22854099_category,SRR22854094_category,SRR22854095_category,SRR22854096_category,SRR23051879_category,SRR23051878_category,SRR23051877_category,SRR23051876_category,SRR23051875_category,SRR23051874_category
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""EPlORYSAT000373610""","""UNCHANGED""","""UNCHANGED""","""UP""","""UP""","""DOWN""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""DOWN""","""UNCHANGED""","""UP""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED"""
"""EPlORYSAT000373621""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""",…,"""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""UNCHANGED""","""

In [5]:
# https://docs.pola.rs/api/python/stable/reference/expressions/api/polars.sum_horizontal.html#polars-sum-horizontal
count = result.with_columns([
    (pl.sum_horizontal([
        (pl.col(col) == "UP").cast(pl.Int32) for col in result.columns if col != "GENEID"
    ])).alias(f"UP{threshold}"),
    (pl.sum_horizontal([
        (pl.col(col) == "DOWN").cast(pl.Int32) for col in result.columns if col != "GENEID"
    ])).alias(f"DOWN{threshold}"),
    pl.sum_horizontal([
        (pl.col(col) == "UNCHANGED").cast(pl.Int32) for col in result.columns if col != "GENEID"
    ]).alias(f"UNCHANGED{threshold}")
]).drop(
    [col for col in result.columns if col != 'GENEID']
).with_columns([
    (pl.col(f"UP{threshold}") - pl.col(f"DOWN{threshold}")).alias(f"HN{threshold}"),
    ((pl.col(f"UP{threshold}") + 1) / (pl.col(f"DOWN{threshold}") + 1)).alias(f"UPcount+1 / DOWNcount+1 {threshold}"),
]).sort(by=f"HN{threshold}", descending=True)


display(count)

GENEID,UP5,DOWN5,UNCHANGED5,HN5,UPcount+1 / DOWNcount+1 5
str,i32,i32,i32,i32,f64
"""Os04g0107900""",263,8,89,255,29.333333
"""Os01g0136100""",253,8,99,245,28.222222
"""Os02g0259900""",246,8,106,238,27.444444
"""Os02g0259850""",245,8,107,237,27.333333
"""Os03g0245800""",248,11,101,237,20.75
…,…,…,…,…,…
"""Os01g0952800""",7,180,173,-173,0.044199
"""Os07g0142100""",11,187,162,-176,0.06383
"""Os03g0307200""",4,184,172,-180,0.027027


In [8]:
# making directory each time
directory = f'../Data/06_HNscore/HN-score_rice_{now.strftime("%y%m%d")}'
os.makedirs(directory, exist_ok=True) # exist_ok=True: if the directory already exists, no error

count.write_csv(f'{directory}/HN-score_rice_{now.strftime("%y%m%d")}_HN{threshold}_all.tsv', separator="\t")

In [7]:
# temp = pd.DataFrame(index=HNratio.index)
# # Upregulated
# thresholds = [1.5, 2,5, 10, 20, 50, 100, 200] # input thresholds here

# for thre in thresholds:
#     # HNratio >= thre
#     up_count = (HNratio >= thre).sum(axis=1)
#     temp[f'up{thre}'] = up_count

#     # HNratio <= 1/thre
#     dn_count = (HNratio <= 1/thre).sum(axis=1)
#     temp[f'dn{thre}'] = dn_count

#     # Unchanged
#     unchanged_count = len(HNratio.columns) - (up_count + dn_count)
#     temp[f'unchanged{thre}'] = unchanged_count

#     # HN-score
#     temp[f'HN{thre}'] = up_count - dn_count


# HN_score_columns = temp.filter(like='HN').columns
# HN_scores = temp[HN_score_columns]

# display(temp, HN_scores)

# example of sorting by HN5
# tempHN5= HN_scores.sort_values(by= "HN5", ascending=False)
# display(tempHN5)